In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
import os
# 한글 지원
os.putenv('NLS_LANG', '.UTF8')

In [3]:
def ipchecker():    
    import urllib 
    import re
    
    pattern = re.compile(b'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    dyndns = urllib.request.urlopen('http://checkip.dyndns.org').read()
    
    wan = re.search(pattern, dyndns).group().decode('utf-8')
    
    return wan

In [4]:
# base db_connect param

local_ip = '200.1.220.251:1521/xe' # 원내에서 접속할 db_ip
outside_ip = '61.35.152.103:1521/xe' # 원밖에서 접속할 db_ip
localhost = 'localhost:1521/xe'

db_id = 'begopa' 
db_pwd = 'begopa1'

academy_ip = '61.35.152.66'
db_ip = local_ip if academy_ip == ipchecker() else outside_ip

# db_ip = localhost # 집에서 본인의 개인 컴퓨터dptj  오라클에 접속할때는

In [5]:
import pickle
# load object ::
# :: file_nm
# :: columns and data_type
with open("./pickle/col2dict.pickle","rb") as fr:
    col2dict = pickle.load(fr)
with open("./pickle/table_nm_set.pickle","rb") as fr:
    table_nm_set = pickle.load(fr)

In [6]:
path = "./sg-data/"
desc_path = path + 'desc/'
col_path = path + 'col/'
origin_path = path + 'origin/'

file_list = os.listdir(origin_path)

In [7]:
# 자치구별 상권 변화지표 db 에서 한글 컬럼으로 가져오기
# oracle_to_df('자치구별 상권변화지표')
def load_to_df(kr_tbl_nm):
    for key, value in table_nm_set.items():
        if kr_tbl_nm == value:
            table_nm = key
            columns = [ arr[0] for arr in col2dict[key] ]
           
    import cx_Oracle
    import pandas as pd
    
    db = cx_Oracle.connect(db_id, db_pwd, db_ip) 
        
    sql = 'select * from ' + table_nm

    df = pd.read_sql(sql=sql, con = db) 
    df.columns = columns
    db.close
    
    return df

import re
import os

file_list = os.listdir(origin_path)
for file in file_list:
    kr_tbl_nm = re.findall('(\d\d).*\((.*)\)',file)
    print(kr_tbl_nm[0][0], kr_tbl_nm[0][1])

00 상권-영역
01 상권-추정유동인구
02 상권-집객시설
03 상권배후지-추정유동인구
04 상권배후지-집객시설
05 상권배후지-상주인구
06 상권-상주인구
07 상권배후지-아파트
08 상권배후지-점포
09 상권-상권변화지표
10 행정동별 상권변화지표
11 상권-추정매출
12 상권-직장인구
13 상권배후지-소득소비
14 상권배후지-직장인구
15 자치구별 상권변화지표
16 상권-아파트
18 상권-점포
19 상권배후지-추정매출


In [8]:
df0=load_to_df( '상권-영역' )
# df0.head()
# df0.info()
df10 = load_to_df('행정동별 상권변화지표')   #10857
# df10.head()
df15=load_to_df( '자치구별 상권변화지표' )
# df15.info()
# df15.head()

In [9]:
df0.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1496 entries, 0 to 1495
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   상권_구분_코드    1496 non-null   object
 1   상권_구분_코드_명  1496 non-null   object
 2   상권_코드       1496 non-null   object
 3   상권_코드_명     1496 non-null   object
 4   엑스좌표_값      1496 non-null   int64 
 5   와이좌표_값      1496 non-null   int64 
 6   시군구_코드      1496 non-null   object
 7   행정동_코드      1496 non-null   object
dtypes: int64(2), object(6)
memory usage: 93.6+ KB


In [10]:
tt=df0[['행정동_코드','시군구_코드']]

In [11]:
df10

,기준_년_분기_코드,행정동_코드,행정동_코드_명,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균
0,2020_1,11740700,둔촌2동,HL,상권축소,113,50,110,54
1,2020_1,11740690,둔촌1동,HH,정체,156,70,110,54
2,2020_1,11740685,길동,LL,다이나믹,104,48,110,54
3,2020_1,11740660,성내3동,HL,상권축소,111,47,110,54
4,2020_1,11740650,성내2동,LL,다이나믹,105,48,110,54
...,...,...,...,...,...,...,...,...,...
10852,2014_1,11110560,평창동,HH,정체,95,50,91,44
10853,2014_1,11110550,부암동,LH,상권확장,90,47,91,44
10854,2014_1,11110540,삼청동,LH,상권확장,77,48,91,44
10855,2014_1,11110530,사직동,HH,정체,103,50,91,44


In [12]:
# set(df10['행정동_코드']) - set(df0['행정동_코드'])

In [13]:
tt1=df10.merge(tt , on='행정동_코드').drop_duplicates()
tt2=tt1.merge(df15 , on=['시군구_코드','기준_년_분기_코드']).drop_duplicates()
tt2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9877 entries, 0 to 9876
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   기준_년_분기_코드        9877 non-null   object
 1   행정동_코드            9877 non-null   object
 2   행정동_코드_명          9877 non-null   object
 3   상권_변화_지표_x        9877 non-null   object
 4   상권_변화_지표_명_x      9877 non-null   object
 5   운영_영업_개월_평균_x     9877 non-null   int64 
 6   폐업_영업_개월_평균_x     9877 non-null   int64 
 7   서울_운영_영업_개월_평균_x  9877 non-null   int64 
 8   서울_폐업_영업_개월_평균_x  9877 non-null   int64 
 9   시군구_코드            9877 non-null   object
 10  시군구_코드_명          9877 non-null   object
 11  상권_변화_지표_y        9877 non-null   object
 12  상권_변화_지표_명_y      9877 non-null   object
 13  운영_영업_개월_평균_y     9877 non-null   int64 
 14  폐업_영업_개월_평균_y     9877 non-null   int64 
 15  서울_운영_영업_개월_평균_y  9877 non-null   int64 
 16  서울_폐업_영업_개월_평균_y  9877 non-null   int64 
dtypes: int64(8), o

In [14]:
tt2.columns

tt2=tt2.rename(columns={'상권_변화_지표_x':'동_상권_변화지표', '상권_변화_지표_명_x':'동_상권_변화지표_명',
                   '운영_영업_개월_평균_x':'동_영업_개월_평균', '폐업_영업_개월_평균_x':'동_폐업_개월_평균', '서울_운영_영업_개월_평균_x':'동_서울_영업_개월_평균',
                   '서울_폐업_영업_개월_평균_x':'동_서울_폐업_개월_평균', '상권_변화_지표_y':'구_상권_변화지표', '상권_변화_지표_명_y':'구_상권_변화지표_명',
                   '운영_영업_개월_평균_y':'구_영업_개월_평균', '폐업_영업_개월_평균_y':'구_폐업_개월_평균', '서울_운영_영업_개월_평균_y':'구_서울_영업_개월_평균',
                   '서울_폐업_영업_개월_평균_y':'구_서울_폐업_개월_평균'})
tt2

,기준_년_분기_코드,행정동_코드,행정동_코드_명,동_상권_변화지표,동_상권_변화지표_명,동_영업_개월_평균,동_폐업_개월_평균,동_서울_영업_개월_평균,동_서울_폐업_개월_평균,시군구_코드,시군구_코드_명,구_상권_변화지표,구_상권_변화지표_명,구_영업_개월_평균,구_폐업_개월_평균,구_서울_영업_개월_평균,구_서울_폐업_개월_평균
0,2020_1,11740700,둔촌2동,HL,상권축소,113,50,110,54,11740,강동구,LL,다이나믹,106,50,110,54
1,2020_1,11740685,길동,LL,다이나믹,104,48,110,54,11740,강동구,LL,다이나믹,106,50,110,54
2,2020_1,11740660,성내3동,HL,상권축소,111,47,110,54,11740,강동구,LL,다이나믹,106,50,110,54
3,2020_1,11740650,성내2동,LL,다이나믹,105,48,110,54,11740,강동구,LL,다이나믹,106,50,110,54
4,2020_1,11740640,성내1동,LL,다이나믹,109,51,110,54,11740,강동구,LL,다이나믹,106,50,110,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9872,2014_3,11110560,평창동,HH,정체,98,51,93,45,11110,종로구,HH,정체,111,52,93,45
9873,2014_3,11110550,부암동,LH,상권확장,92,48,93,45,11110,종로구,HH,정체,111,52,93,45
9874,2014_3,11110540,삼청동,LH,상권확장,76,50,93,45,11110,종로구,HH,정체,111,52,93,45
9875,2014_3,11110530,사직동,HH,정체,104,51,93,45,11110,종로구,HH,정체,111,52,93,45


In [15]:
tt2.columns
# ['기준_년_분기_코드', '행정동_코드', '행정동_코드_명', '동_상권_변화지표', '동_상권_변화지표_명',
#        '동_영업_개월_평균', '동_폐업_개월_평균', '동_서울_영업_개월_평균', '동_서울_폐업_개월_평균', '시군구_코드',
#        '시군구_코드_명', '구_상권_변화지표', '구_상권_변화지표_명', '구_영업_개월_평균', '구_폐업_개월_평균',
#        '구_서울_영업_개월_평균', '구_서울_폐업_개월_평균']



Index(['기준_년_분기_코드', '행정동_코드', '행정동_코드_명', '동_상권_변화지표', '동_상권_변화지표_명',
       '동_영업_개월_평균', '동_폐업_개월_평균', '동_서울_영업_개월_평균', '동_서울_폐업_개월_평균', '시군구_코드',
       '시군구_코드_명', '구_상권_변화지표', '구_상권_변화지표_명', '구_영업_개월_평균', '구_폐업_개월_평균',
       '구_서울_영업_개월_평균', '구_서울_폐업_개월_평균'],
      dtype='object')

In [16]:
# tt2[['기준_년_분기_코드', '행정동_코드', '행정동_코드_명','시군구_코드','시군구_코드_명','동_상권_변화지표', '동_상권_변화지표_명',
#        '동_영업_개월_평균', '동_폐업_개월_평균', '동_서울_영업_개월_평균', '동_서울_폐업_개월_평균', 
#        '구_상권_변화지표', '구_상권_변화지표_명', '구_영업_개월_평균', '구_폐업_개월_평균',
#        '구_서울_영업_개월_평균', '구_서울_폐업_개월_평균']]

In [17]:
def multi_columns_merging(sg, sg_hl):
    a_df = load_to_df( sg )
    b_df = load_to_df( sg_hl )
    all_df = pd.merge(left=a_df, right=b_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
#     del a_df, b_df
    all_df2 = all_df.drop(all_df.filter(regex='연령대|시간대'),axis=1)
    return all_df2.dropna(axis=0)

def multi_columns_merging2(df, sg):
    a_df = load_to_df( sg )
    all_df = pd.merge(left=df, right=a_df, how='left', \
                  on=['기준_년_분기_코드','상권_코드'], sort=False)
#     del a_df, b_df
    all_df2 = all_df.drop(all_df.filter(regex='연령대|시간대'),axis=1)
    return all_df2.dropna(axis=0)



In [18]:
p1 = multi_columns_merging('상권-추정유동인구','상권-상주인구')

In [19]:
p2=multi_columns_merging2(p1, '상권-직장인구')

In [20]:
target3=['월요','화요','수요','목요','금요']
target4=['토요','일요']
def add_week2(df,target3,target4):
    df['주중_유동인구_수']=df.filter(regex='|'.join(target3)).sum(axis=1)
    df['주말_유동인구_수']=df.filter(regex='|'.join(target4)).sum(axis=1)
    df_a=df.drop(df.filter(regex='|'.join(target3+target4)),axis=1)
    return df_a

# hl을 먼저 합해라!

In [21]:
p3=add_week2(p2,target3,target4)
p3.head()

,기준_년_분기_코드,상권_코드,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,총_상주인구_수,남성_상주인구_수,여성_상주인구_수,총_가구_수,아파트_가구_수,비_아파트_가구_수,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,주중_유동인구_수,주말_유동인구_수
0,2019_1,1000001,118266,61268,56997,1806.0,851.0,955.0,785.0,0.0,785.0,348.0,142.0,206.0,86815,31451
1,2019_1,1000002,246619,147358,99261,841.0,383.0,458.0,775.0,8.0,767.0,486.0,254.0,232.0,156385,90234
2,2019_1,1000003,144480,80073,64407,1214.0,682.0,532.0,805.0,0.0,805.0,4822.0,2188.0,2634.0,107665,36815
3,2019_1,1000012,166390,70339,96051,2586.0,1163.0,1423.0,1087.0,111.0,976.0,101.0,56.0,45.0,121377,45018
4,2019_1,1000013,65518,33437,32080,249.0,147.0,102.0,166.0,0.0,166.0,615.0,353.0,262.0,46747,18769


In [22]:
p3.columns

Index(['기준_년_분기_코드', '상권_코드', '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수', '총_상주인구_수',
       '남성_상주인구_수', '여성_상주인구_수', '총_가구_수', '아파트_가구_수', '비_아파트_가구_수',
       '총_직장_인구_수', '남성_직장_인구_수', '여성_직장_인구_수', '주중_유동인구_수', '주말_유동인구_수'],
      dtype='object')

In [23]:
p3=p3[['기준_년_분기_코드', '상권_코드','남성_유동인구_수', '여성_유동인구_수',
        '주중_유동인구_수', '주말_유동인구_수', '남성_상주인구_수', '여성_상주인구_수','남성_직장_인구_수', '여성_직장_인구_수']]

In [24]:
p3  # 유동, 상주, 직장

,기준_년_분기_코드,상권_코드,남성_유동인구_수,여성_유동인구_수,주중_유동인구_수,주말_유동인구_수,남성_상주인구_수,여성_상주인구_수,남성_직장_인구_수,여성_직장_인구_수
0,2019_1,1000001,61268,56997,86815,31451,851.0,955.0,142.0,206.0
1,2019_1,1000002,147358,99261,156385,90234,383.0,458.0,254.0,232.0
2,2019_1,1000003,80073,64407,107665,36815,682.0,532.0,2188.0,2634.0
3,2019_1,1000012,70339,96051,121377,45018,1163.0,1423.0,56.0,45.0
4,2019_1,1000013,33437,32080,46747,18769,147.0,102.0,353.0,262.0
...,...,...,...,...,...,...,...,...,...,...
2957,2019_2,1001325,60736,48646,82138,27244,41.0,17.0,7.0,4.0
2958,2019_2,1001379,51637,60622,82726,29532,153.0,155.0,7.0,44.0
2959,2019_2,1001380,70806,63253,88801,45257,255.0,255.0,53.0,18.0
2960,2019_2,1001381,18328,20862,27663,11527,195.0,189.0,9.0,5.0


In [25]:
df18=load_to_df('상권-점포')
df18

,기준_년_분기_코드,상권_코드,서비스_업종_코드,서비스_업종_코드_명,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수
0,2019_2,1001496,CS300043,전자상거래업,7,7,0,0,14,1,0
1,2019_2,1001496,CS300042,주유소,1,1,0,0,0,0,0
2,2019_2,1001496,CS300041,예술품,9,9,0,0,0,0,0
3,2019_2,1001496,CS300040,재생용품 판매점,0,0,0,0,0,0,0
4,2019_2,1001496,CS300038,자동차부품,10,10,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
210934,2019_1,1000019,CS300032,가전제품,0,0,0,0,0,0,0
210935,2019_1,1000019,CS300031,가구,1,1,0,0,0,0,0
210936,2019_1,1000019,CS300028,화초,1,1,0,0,0,0,0
210937,2019_1,1000019,CS300027,섬유제품,0,0,0,0,0,0,0


In [26]:
df11=load_to_df('상권-추정매출')

In [27]:
df11 = df11.rename(columns={'점포수':'점포_수'})
df11.columns

Index(['기준_년_분기_코드', '상권_코드', '서비스_업종_코드', '서비스_업종_코드_명', '점포_수', '당월_매출_금액',
       '당월_매출_건수', '주중_매출_비율', '주말_매출_비율', '월요일_매출_비율', '화요일_매출_비율',
       '수요일_매출_비율', '목요일_매출_비율', '금요일_매출_비율', '토요일_매출_비율', '일요일_매출_비율',
       '시간대_00~06_매출_비율', '시간대_06~11_매출_비율', '시간대_11~14_매출_비율',
       '시간대_14~17_매출_비율', '시간대_17~21_매출_비율', '시간대_21~24_매출_비율', '남성_매출_비율',
       '여성_매출_비율', '연령대_10_매출_비율', '연령대_20_매출_비율', '연령대_30_매출_비율',
       '연령대_40_매출_비율', '연령대_50_매출_비율', '연령대_60_이상_매출_비율', '주중_매출_금액',
       '주말_매출_금액', '월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액',
       '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액', '시간대_00~06_매출_금액',
       '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액',
       '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액', '여성_매출_금액',
       '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액',
       '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '주중_매출_건수', '주말_매출_건수', '월요일_매출_건수',
       '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수',
       '일요일_매

In [28]:
word1=['월요일_매출_건수','화요일_매출_건수','수요일_매출_건수','목요일_매출_건수','금요일_매출_건수']
word2=['토요일_매출_건수','일요일_매출_건수']
word3=['월요일_매출_금액','화요일_매출_금액','수요일_매출_금액','목요일_매출_금액','금요일_매출_금액']
word4=['토요일_매출_금액','일요일_매출_금액']

df11['주중_매출_건수']=df11.filter(regex='|'.join(word1)).sum(axis=1)
df11['주말_매출_건수']=df11.filter(regex='|'.join(word2)).sum(axis=1)
df11['주중_매출_금액']=df11.filter(regex='|'.join(word3)).sum(axis=1)
df11['주말_매출_금액']=df11.filter(regex='|'.join(word4)).sum(axis=1)
df11=df11.drop(df11.filter(regex='|'.join(word1+word2+word3+word4)),axis=1)

In [29]:
df11=df11.drop(df11.filter(regex='연령대|시간대'),axis=1)
df11.columns
df11_1 = df11[['기준_년_분기_코드', '상권_코드', '서비스_업종_코드', '서비스_업종_코드_명', '점포_수', '당월_매출_금액',
               '당월_매출_건수','주중_매출_금액', '주말_매출_금액', 
              '남성_매출_금액','여성_매출_금액']]

In [30]:
s1=df11_1.merge(df18, on=['기준_년_분기_코드','상권_코드','서비스_업종_코드','서비스_업종_코드_명','점포_수']).drop_duplicates()

In [31]:
# s1  #60360

In [32]:
p3.columns

Index(['기준_년_분기_코드', '상권_코드', '남성_유동인구_수', '여성_유동인구_수', '주중_유동인구_수',
       '주말_유동인구_수', '남성_상주인구_수', '여성_상주인구_수', '남성_직장_인구_수', '여성_직장_인구_수'],
      dtype='object')

In [33]:
p3.to_csv('all-people.csv', index=False)
s1.to_csv('sales.csv', index=False)

In [34]:
s1.columns

Index(['기준_년_분기_코드', '상권_코드', '서비스_업종_코드', '서비스_업종_코드_명', '점포_수', '당월_매출_금액',
       '당월_매출_건수', '주중_매출_금액', '주말_매출_금액', '남성_매출_금액', '여성_매출_금액',
       '유사_업종_점포_수', '개업_율', '개업_점포_수', '폐업_률', '폐업_점포_수', '프랜차이즈_점포_수'],
      dtype='object')

In [35]:
# s1.merge(p3, on=['기준_년_분기_코드', '상권_코드']).drop_duplicates()

In [36]:
tt2.to_csv('change.csv', index=False)